In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

In [2]:
train_dir = r'C:\Users\naomi\upx_cinco\Scripts\dataset\treinamento'
test_dir = r'C:\Users\naomi\upx_cinco\Scripts\dataset\teste'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalização
    rotation_range=20,       # Rotação aleatória
    width_shift_range=0.2,   # Deslocamento horizontal
    height_shift_range=0.2,  # Deslocamento vertical
    
    shear_range=0.2,        # Cortes
    zoom_range=0.2,         # Zoom
    horizontal_flip=True,    # Espelhar horizontalmente
    fill_mode='nearest'      # Modo de preenchimento
)

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)  

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),  # Redimensionar as imagens
    batch_size=32,
    class_mode='categorical'  # Se você tem múltiplas classes
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical'
)

Found 6 images belonging to 3 classes.
Found 6 images belonging to 3 classes.


In [6]:
model = models.Sequential()
model.add(layers.Input(shape=(32, 32, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(train_generator.class_indices), activation='softmax'))  # Número de classes

# Compilar o modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3333 - loss: 1.1198

C:\Users\naomi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3333 - loss: 1.1198 - val_accuracy: 0.3333 - val_loss: 1.0637
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.6667 - loss: 1.0628 - val_accuracy: 1.0000 - val_loss: 1.0137
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.5000 - loss: 1.0280 - val_accuracy: 1.0000 - val_loss: 0.9644
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.8333 - loss: 1.0017 - val_accuracy: 1.0000 - val_loss: 0.9021
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 1.0000 - loss: 0.9227 - val_accuracy: 1.0000 - val_loss: 0.8232
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 1.0000 - loss: 0.8654 - val_accuracy: 1.0000 - val_loss: 0.7329
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.8333 - loss: 0.7859 - val_accuracy: 1.0000 - val_loss: 0.6415
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 1.0000 - loss: 0.7165 - val_accuracy: 1.0000 - val_loss: 0.5667
Epoch 

In [7]:
# Avaliar o modelo
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'\nAcurácia no conjunto de teste: {test_acc*100:.2f}%')

# Para prever uma nova imagem
from tensorflow.keras.preprocessing import image
import numpy as np

# Função para carregar e preprocessar uma nova imagem
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(32, 32))  # Redimensionar
    img_array = image.img_to_array(img) / 255.0  # Normalizar
    img_array = np.expand_dims(img_array, axis=0)  # Adicionar dimensão do lote
    return img_array

# Exemplo de previsão
img_path = r'C:\Users\naomi\OneDrive\Pictures\WhatsApp Image 2024-10-01 at 19.06.33.jpeg'
img = load_and_preprocess_image(img_path)
predictions = model.predict(img)
predicted_class = np.argmax(predictions)

# Mostrar o resultado
print(f'Classe prevista: {list(train_generator.class_indices.keys())[predicted_class]}')

confidence = np.max(predictions) * 100  # Multiplicar por 100 para obter a porcentagem
print(f'Confiança: {confidence:.2f}%')

1/1 - 0s - 67ms/step - accuracy: 0.8333 - loss: 0.5407

Acurácia no conjunto de teste: 83.33%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Classe prevista: peixes
Confiança: 97.22%


In [9]:
model.save('modelo_classificacao_animais.keras')

In [10]:
model.export('saved_model_dir')  # Salva o modelo no formato SavedModel

INFO:tensorflow:Assets written to: saved_model_dir\assets


INFO:tensorflow:Assets written to: saved_model_dir\assets


Saved artifact at 'saved_model_dir'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1324343845392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1324561621520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1324343844816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1324343845200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1324563173008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1324563173584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1324563173200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1324563174736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1324563172432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1324563174160: TensorSpec(shape=(), dtype=tf.resource, name=None)
